<a href="https://colab.research.google.com/github/kushagra-joshi25/Route.py/blob/main/Copy_of_route.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
City_Gps = {}
Road_Seg = pd.read_csv('https://raw.githubusercontent.com/rakshith91/CityNavigation/master/road-segments.txt', delimiter=' ',
                       names=['City1', 'City2', 'Dist', 'Speed', 'HighWay'])


# replace the NaN's and O's

def smean(sumx):
    return ((sum(Road_Seg['Speed']))/(len(Road_Seg['Speed'])))


def dmean(sumx):
    return ((sum(Road_Seg['Speed']))/(len(Road_Seg['Speed'])))

data = open('city-gps.txt', "r")
# print(data.read())
for line in data:
    line = list(map(str, line.split()))
    key, val = line[0], line[1:]
    City_Gps[key] = val

Latitude=[]
Longitude=[]
City=[]
for row in City_Gps:
  City.append(row)
  Latitude.append(City_Gps[row][0])
  Longitude.append(City_Gps[row][1])
data=pd.read_csv('https://raw.githubusercontent.com/rakshith91/CityNavigation/master/road-segments.txt',delimiter=' ',names=['City1', 'City2', 'Dist', 'Speed', 'HighWay'])
data['Dist'] = data['Dist'].fillna(0)
data[data["Dist"] == 0] = data[data["Dist"]== 0].replace(0, dmean(data["Dist"]))
#df_new=pd.DataFrame(list(zip(Latitude, Longitude)),columns =['Lat','Long'],index=City)

In [1]:
import pandas as pd
import math
import sys
from collections import deque
import heapq
import itertools
import copy
import random as rn
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from numpy.random import choice as np_choice
import plotly.express as px
# from collections import defaultdict
total_distance = 0
total_time = 0

# ----------------------
closed = []

# Previus dictionary for finding the path
previous = {}
print("Enter route type")
route = input()
# route = "antcolony"
if route=='bfs' or route=='dfs' or route=='astar' :
  print("Enter Starting City")
  city = input()
  print("Enter Ending City")
  city2 = input()
  print("Enter cost type")
  cost_type = input()




# Replacing the NAN's and 0's in the Speed column with mean of the Speed column.
Road_Seg['Speed'] = Road_Seg['Speed'].fillna(0)
Road_Seg[Road_Seg["Speed"] == 0] = Road_Seg[Road_Seg["Speed"]
                                            == 0].replace(0, smean(Road_Seg["Speed"]))


# Replacing the NAN's and 0's in the Speed column with mean of the Dist column.

Road_Seg['Dist'] = Road_Seg['Dist'].fillna(0)
Road_Seg[Road_Seg["Dist"] == 0] = Road_Seg[Road_Seg["Dist"]
                                           == 0].replace(0, dmean(Road_Seg["Dist"]))


pq_fringe = []
pq_finder = {}
REMOVED = '<removed-task>'
counter = itertools.count()


def add_task(task, priority=0):
    'Add a new task or update the priority of an existing task'
    if task in pq_finder:
        remove_task(task)
    count = next(counter)
    entry = [priority, count, task]
    pq_finder[task] = entry
    heapq.heappush(pq_fringe, entry)


def remove_task(task):
    'Mark an existing task as REMOVED.  Raise KeyError if not found.'
    entry = pq_finder.pop(task)
    entry[-1] = REMOVED


def pop_task():
    'Remove and return the lowest priority task. Raise KeyError if empty.'
    while pq_fringe:
        priority, count, task = heapq.heappop(pq_fringe)
        if task is not REMOVED:
            del pq_finder[task]
            return task
    raise KeyError('pop from an empty priority queue')
# HeapQ

# ------------------------------------------------------------------------------------------------------------------------

# --------------------------------##--------------------------------------------------------
# Distance using the lat's and long's using havershine
# Modified version of finding havershine distance  https://gist.github.com/rochacbruno/2883505


def distance_haversine(start_city, goal_city):

    if (start_city in City_Gps) and (goal_city in City_Gps):
        lat1, long1 = (City_Gps[start_city])
        lat2, long2 = City_Gps[goal_city]

        lat1 = float(lat1)
        long1 = float(long1)

        lat2 = float(lat2)
        long2 = float(long2)

        radius = 3596  # radius of earth in  miles
        dlat = math.radians(lat2 - lat1)
        dlon = math.radians(long2 - long1)
        a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) \
            * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(
            dlon / 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        d = radius * c
        if cost_type == "distance":
            return d
        elif cost_type == "time":
            return (d/65)-2
        elif cost_type == "segments":
            return 1

    return 99999.0


# --------------------------------##--------------------------------------------------------
# Finding time
Road_Seg["time"] = Road_Seg["Dist"]/Road_Seg["Speed"]


class Graph:
    # Initiating graph

    def __init__(self):
        self.__graph_dict = {}
        self.__edge_dict = {}

    def add_node(self, fromc, toc, dist, time):
        # new_node = toc
        if fromc not in self.__graph_dict:
            self.__graph_dict[fromc] = set()
            self.__graph_dict[fromc].add(toc)
        else:
            self.__graph_dict[fromc].add(toc)
        if toc not in self.__graph_dict:
            self.__graph_dict[toc] = set()
            self.__graph_dict[toc].add(fromc)
        else:
            self.__graph_dict[toc].add(fromc)
        edge_key = frozenset((fromc, toc))
        if edge_key not in self.__edge_dict:
            # self.__edge_dict[edge_key] = []
            self.__edge_dict[edge_key] = [dist, time]

    def print_graph(self):
        print(self.__graph_dict)

    def get_graph(self):
        return self.__graph_dict

    def get_edges(self):
        return self.__edge_dict


graph = Graph()

# storing both the from and to city names , distances, time in the graph
for i in range(len(Road_Seg)):
    graph.add_node(Road_Seg['City1'][i], Road_Seg['City2'][i], Road_Seg['Dist'][i], Road_Seg['time'][i])

test_graph = graph.get_graph()
edges = graph.get_edges()


def cost_from_start(path, cost_type):
    cost = 0

    if cost_type == 'distance':
        for i in range(len(path) - 1):
            edge = frozenset((path[i], path[i + 1]))
            cost += edges[edge][0]

    if cost_type == 'time':
        for i in range(len(path) - 1):
            edge = frozenset((path[i], path[i + 1]))
            cost += edges[edge][1]

    if cost_type == 'segments':
        cost = len(path)

    return cost


def dfs(graph, start, goal):
    dfs_stack = [(start, [start])]
    while dfs_stack:
        (node, path) = dfs_stack.pop()
        for next in graph[node] - set(path):
            if next == goal:
                return path + [next]
            else:
                dfs_stack.append((next, path + [next]))
    return None


def bfs(graph, start, goal):
    bfs_queue = deque([(start, [start])])
    while bfs_queue:
        (node, path) = bfs_queue.popleft()
        for next in graph[node] - set(path):
            if next == goal:
                return path + [next]
            else:
                bfs_queue.append((next, path + [next]))
    return None


def a_star(graph, start, goal, cost_type):

    previous[start] = [start, [start], 0]
    add_task(start, distance_haversine(start, goal))
    visited = []
    while len(pq_fringe) > 0:

        curr_state = pop_task()
        closed.append(curr_state)

        visited.append(curr_state)

        if curr_state == goal:
            return previous[curr_state][1]


#        print path
        for s in graph[curr_state] - set(visited):
            start_to_end_path = copy.deepcopy(previous[curr_state][1])
            start_to_end_path.append(s)

            heuristic = distance_haversine(s, goal)
            previous[s] = [s, start_to_end_path, heuristic]
            cost_start_to_succ = cost_from_start(start_to_end_path, cost_type)

            if(route == "uniform"):
                cost_start_to_succ = 1+len(previous[curr_state][1])
                heuristic = 0

            f = cost_start_to_succ + heuristic

            if(s in closed):
                continue
            if s in pq_finder:
                entry = pq_finder[s]
                if entry[0] > f:
                    remove_task(s)

            if not(s in pq_finder):
                add_task(s, f)
                previous[s] = [curr_state, start_to_end_path, cost_start_to_succ]


class AntColony(object):


    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1, flag=0):
        """
        Args:
            distances (2D numpy.array): Square matrix of distances. Diagonal is assumed to be np.inf.
            n_ants (int): Number of ants running per iteration
            n_best (int): Number of best ants who deposit pheromone
            n_iteration (int): Number of iterations
            decay (float): Rate it which pheromone decays. The pheromone value is multiplied by decay, so 0.95 will lead to decay, 0.5 to much faster decay.
            alpha (int or float): exponent on pheromone, higher alpha gives pheromone more weight. Default=1
            beta (int or float): exponent on distance, higher beta give distance more weight. Default=1
            flag (int or boolean): signifies whether paths will be printed or not
        Example:
            ant_colony = AntColony(german_distances, 100, 20, 2000, 0.95, alpha=1, beta=2)
        """
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.flag = flag

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if(self.flag==1):
                print (i,shortest_path,sep="    ") #Displaying shortest path of each iteration
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path
            self.pheromone = self.pheromone * self.decay
        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = set()
        visited.add(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.add(move)
        path.append((prev, start)) # going back to where we started
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        pheromone[list(visited)] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move

def print_path():
    total_distance = 0
    total_time = 0
    for i in range(len(path) - 1):
        edge = frozenset((path[i], path[i + 1]))
        total_distance += edges[edge][0]
        total_time += edges[edge][1]
        s = ""
        for p in path:
            s += p+" "
    print(total_distance, total_time, s) # Printing the required format

def plotbest(max_ant,iter):
    #Finding the best Number of ants and Number of best ants value for the found shortest path
    a=[]
    b=[]
    c=[]
    for i in range(10,max_ant+1,10):# checking in intervals of 10
      for j in range(int(i/10),i,int(i/5)):# checking in intervals of i/5
        ant = AntColony(distance_matrix_array, i, j, iter, 0.95, alpha=1, beta=1,flag=0)
        s_path = ant.run()
        a.append(i)
        b.append(j)
        c.append(s_path[-1])
    q=-1
    for k in c:
      q=q+1
      if(k==min(c)):
        print("\nNumber of ants:",a[q],"\nNumber of best ants:",b[q],"\nShortest Path length:",c[q])
        break

    df2 = pd.DataFrame(list(zip(a, b, c)),columns =['N_ANTS', 'N_BEST','S_PATH'])
    print(df2)
    fig = px.scatter_3d(df2, x='N_ANTS', y='N_BEST', z='S_PATH', title="Finding the best Number of ants and Number of best ants  value")
    fig.show()

    #Finding the best alpha beta value for the found shortest path
    l=[]
    m=[]
    n=[]
    z=np.arange(0.5, 4, 0.5)
    for i in z:
      for j in z:
        ant = AntColony(distance_matrix_array, a[q], b[q], iter, 0.95, alpha=i, beta=j,flag=0)
        s_path = ant.run()
        l.append(i)
        m.append(j)
        n.append(s_path[-1])
    p=-1
    for k in n:
      p=p+1
      if(k==min(n)):
        print("\nAlpha:",l[p],"\nBeta:",m[p],"\nShortest Path length:",n[p])
        break
    df3 = pd.DataFrame(list(zip(l, m, n)),columns =['Alpha', 'Beta','S_PATH'])
    print(df3)
    fig2 = px.scatter_3d(df3, x='Alpha', y='Beta', z='S_PATH', title="Finding the best alpha beta value")
    fig2.show()
    return a[q],b[q],l[p],m[p]


# Routing based on the algorithm value
if route == 'bfs':
    path = bfs(test_graph, sys.argv[1], sys.argv[2])
elif route == 'dfs':
    path = dfs(test_graph, sys.argv[1], sys.argv[2])
elif route == 'astar' or route == 'uniform':
    path = a_star(test_graph, city, city2, cost_type)
elif route == 'antcolony':
    print('Enter State')
    State=input()
    Findings=[]
    for i in City:
      if(i.find(State)!=-1):
        Findings.append(i)


    df = pd.DataFrame(list(zip(City, Latitude, Longitude)),columns =['City', 'Lat','Long'])
    df_ant=pd.DataFrame(list(zip(Latitude, Longitude)),columns =['Lat','Long'],index=City)
    df_ant = df[df['City'].isin(Findings)]
    df_ant_new = df_ant.set_index('City')
    df_ant_new.index.names = [None]
    df_ant_new=df_ant_new.astype(float)
    distance_matrix=pd.DataFrame(distance_matrix(df_ant_new.values, df_ant_new.values), index=df_ant_new.index, columns=df_ant_new.index)
    distance_matrix[distance_matrix == 0] = distance_matrix[distance_matrix== 0].replace(0, dmean(data["Dist"]))
    distance_matrix_array=distance_matrix.to_numpy()
    np.fill_diagonal(distance_matrix_array,np.inf)
    print("Finding the best combination of number of ants and number of best ants and alpha beta values for shortest path distance")
    n_ant, n_best_ant, alpha, beta=plotbest(100,80)
    ant_colony = AntColony(distance_matrix_array, n_ant, n_best_ant, 200, 0.95, alpha=alpha, beta=beta,flag=1) #increasing number of iteration for better result
    shortest_path = ant_colony.run()
    print ("All shortest path nodes : {}".format(shortest_path))
    print ("Shortest path length: {}".format(shortest_path[-1]))
    path=[]
    for i in range(len(shortest_path[0])):
      path.append(df_ant['City'].iloc[shortest_path[0][i][1]])

if route=='bfs' or route=='dfs' or route=='astar' :
  if path is None :
    print('No path found')
  else:
    print_path()

Enter route type


KeyboardInterrupt: Interrupted by user

In [ ]:
distance_matrix

<function scipy.spatial.kdtree.distance_matrix>

In [ ]:
#path
df_ant_new

NameError: ignored

\Abingdon,_Virginia
Deerwood,_Minnesota

# New Section

In [ ]:
print('Starting Point:'+ path[0])
print('Ending Point:'+path[-1])

Starting Point:Albertville,_Alabama
Ending Point:Albion,_Rhode_Island


In [ ]:
df1 = df[df['City'].isin(path)]
df1

NameError: ignored

In [ ]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster


In [ ]:
#df1['Lat'] = df['Lat'].astype(float, errors = 'raise')
#df1['Long'] = df['Long'].astype(float, errors = 'raise')
#df1.dtypes

In [ ]:
pointss = []
for rows in path:
    pointss.append([float(City_Gps[rows][0]), float(City_Gps[rows][1])])

print(pointss)


[[34.2675937, -86.2088669], [34.3581473, -86.2947041], [34.6723069, -86.0341463], [35.0478571, -85.6719138], [35.3714575, -85.3905209], [35.6056231, -85.1888498], [35.4939587, -85.0124535], [35.4428512, -84.5929898], [35.4245183, -84.4874289], [35.7564719, -83.9704593], [35.8217485, -84.0118513], [35.9606384, -83.9207392], [35.8681455, -83.561835], [35.9745743, -83.6270896], [36.0153693, -83.4148895], [36.1500934, -83.2829452], [36.2570407, -83.0859967], [36.1631574, -82.8309861], [36.3134397, -82.3534727], [36.5951059, -82.1887444], [36.4745636, -81.804838], [36.6337272, -81.7837303], [36.6223478, -81.150923], [36.9484528, -81.084811], [36.9470648, -80.9414692], [37.1056784, -80.6853433], [37.1298517, -80.4089389], [37.2709704, -79.9414266], [37.7840208, -79.4428157], [38.149576, -79.0716958], [38.0684692, -78.8894682], [38.4079042, -78.6236277], [38.665397, -78.4594522], [38.8403909, -77.4288769], [39.0570514, -78.1036099], [39.0931617, -78.0594426], [39.1517713, -77.9822193], [39.36

In [ ]:
#points = []
#for idx,rows in df1.iterrows():
#    points.append([rows['Lat'], rows['Long']])
#print(points)
#df2=df1.drop(['City'], axis=1)
#points = df2.to_records(index=False)


In [ ]:

m_2 = folium.Map(location=[34.1620404, -86.4755431], tiles='openstreetmap', zoom_start=8)
count=0
#for idx, row in df1.iterrows():
#    folium.Marker(location=[row['Lat'],row['Long']],popup=row['City']).add_to(m_2)
i=0
for each in pointss:
    folium.Marker(each,popup=path[i]).add_to(m_2)
    i=i+1
folium.PolyLine(pointss, color="red", weight=5, opacity=1).add_to(m_2)
m_2